## IoU計算関数
(Intersection over Union)
$$
    IoU = \frac{Intersection Area}{Union Area}
$$



![IoU計算のイメージ図](IoU_image.png)

In [4]:
# boxの仕様：[左上のx座標, 左上のy座標, 右下のx座標, 右下のy座標] 
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

出力結果を模擬したお試しcsvファイルを読み込んで、  
アノテーションに見せかけたbboxとのIoU計算を繰り返して、  
閾値以上で最大のものを出力するようにする

In [10]:
import pandas as pd

def read_csv_with_pandas(filename):

    # CSVファイルをDataFrameに読み込む
    data = pd.read_csv(filename)
    
    return data

# テスト用: CSVを読み込み
csv_data = read_csv_with_pandas("test_data.csv")



In [11]:
csv_data

,frame,ID,min_x,max_x,min_y,max_y
0,1,1,135,283,62,69
1,1,2,253,500,58,136
2,1,3,256,271,85,166
3,2,1,302,414,68,134
4,2,2,216,525,24,105
5,2,3,274,483,21,56
6,2,4,153,446,67,82
7,2,5,229,306,88,172
8,3,1,269,588,66,93
9,3,2,198,283,49,82


In [12]:
#アノテーションに見せかけたbbox
annote_bbox = [250,200, 500, 250]

In [13]:
#アノテーションと検出矩形のIoUを計算して閾値より大きい中での最大値を取る矩形とマッチングさせる
frameNow = 1
frame_data = csv_data[csv_data["frame"] == frameNow]
print(frame_data)

   frame  ID  min_x  max_x  min_y  max_y
0      1   1    135    283     62     69
1      1   2    253    500     58    136
2      1   3    256    271     85    166


In [19]:
# frame_dataからbboxを作成する
def createBboxFromTrace(trace):
    bbox = []
    bbox = (trace["min_x"], trace["min_y"], trace["max_x"],trace["max_y"])

    return bbox

In [30]:
# trace数だけ繰り返してアノテとのIoUを計算
traceFrameNum = len(frame_data)
iouMax = 0
idMax = 9999 # 9999は無効値とする
for i in range(traceFrameNum):
    trace = frame_data.iloc[i]
    traceBbox = createBboxFromTrace(trace)
    iouScore = calculate_iou(traceBbox, annote_bbox)

    if(iouMax < iouScore):
        iouMax = iouScore
        idMax = trace['ID']

コード作成用のお試しcsvファイルを作るやつ

In [5]:
import csv
import random

def generate_test_csv(filename, num_frames, max_objects_per_frame):
    """
    テスト用のCSVファイルを生成する。

    Args:
        filename (str): 出力CSVファイル名
        num_frames (int): フレーム数
        max_objects_per_frame (int): 1フレームあたりの最大対象物数
    """
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # ヘッダーを書き込む
        writer.writerow(["frame", "ID", "min_x", "max_x", "min_y", "max_y"])
        
        # フレームごとにデータを生成
        for frame in range(1, num_frames + 1):
            num_objects = random.randint(1, max_objects_per_frame)  # 対象物の数をランダムに決定
            for obj_id in range(1, num_objects + 1):
                min_x = random.randint(0, 332)
                max_x = min_x + random.randint(1, 332)
                min_y = random.randint(0, 90)
                max_y = min_y + random.randint(1, 90)
                
                # 行を書き込む
                writer.writerow([frame, obj_id, min_x, max_x, min_y, max_y])

# テスト用CSVファイルを生成
generate_test_csv("test_data.csv", num_frames=10, max_objects_per_frame=5)
